# Fieldmaps OK?

This notebook shows how we check if fieldmaps are defined for the data set. There are two approaches:

1. Look at the func/dwi/perf folders; do they have fieldmaps?
2. Look at the fmap folder; doe the files they point to exist?

In [1]:
import glob
import re
import pathlib

In [56]:
# USE THIS BEFORE TESTING! 
import sys 
sys.path.append("..")
from pathlib import Path 
import shutil
import os

from pkg_resources import resource_filename as pkgrf 

# returns stirng path to testdata
TEST_DATA = pkgrf("cubids", "testdata")

# should give you the full path 
tmp_path = Path().resolve()
#print(tmp_path)

# dest path
data_root = tmp_path / "testdata"

# ensure path does not already exist in cwd
if data_root.exists():
    shutil.rmtree(str(data_root))

# equivalent of command line "cp"
cwd = shutil.copytree(TEST_DATA, str(data_root))

In [37]:
# get a list of fieldmaps:

fmaps = pathlib.Path(data_root).rglob("*fmap/*.json")

fmaps = [x for x in fmaps]

fmaps[:10]

[PosixPath('/Users/ttapera/BBL/Projects/CuBIDS/notebooks/testdata/complete/sub-01/ses-phdiff/fmap/sub-01_ses-phdiff_acq-v4_phasediff.json'),
 PosixPath('/Users/ttapera/BBL/Projects/CuBIDS/notebooks/testdata/complete/sub-01/ses-phdiff/fmap/sub-01_ses-phdiff_acq-v4_magnitude2.json'),
 PosixPath('/Users/ttapera/BBL/Projects/CuBIDS/notebooks/testdata/complete/sub-01/ses-phdiff/fmap/sub-01_ses-phdiff_dir-PA_epi.json'),
 PosixPath('/Users/ttapera/BBL/Projects/CuBIDS/notebooks/testdata/complete/sub-01/ses-phdiff/fmap/sub-01_ses-phdiff_acq-v4_magnitude1.json'),
 PosixPath('/Users/ttapera/BBL/Projects/CuBIDS/notebooks/testdata/complete/sub-02/ses-phdiff/fmap/sub-02_ses-phdiff_acq-v4_phasediff.json'),
 PosixPath('/Users/ttapera/BBL/Projects/CuBIDS/notebooks/testdata/complete/sub-02/ses-phdiff/fmap/sub-02_ses-phdiff_acq-v4_magnitude2.json'),
 PosixPath('/Users/ttapera/BBL/Projects/CuBIDS/notebooks/testdata/complete/sub-02/ses-phdiff/fmap/sub-02_ses-phdiff_acq-v4_magnitude1.json'),
 PosixPath('/Us

In [38]:
import json

def read_intendedfor(path):
    
    with open(str(path), 'r') as infile:
        data = json.load(infile)
        
    return data.get('IntendedFor')

In [39]:
read_intendedfor(fmaps[0])

['ses-phdiff/dwi/sub-01_ses-phdiff_acq-HASC55AP_dwi.nii.gz']

Map each fmap to its array of intendedfor's:

In [54]:
mapping = {}

for fm in fmaps:
    
    intfor = read_intendedfor(fm)
    
    mapping[str(fm)] = intfor

In the first method, we just list whether or not the fieldmap has files that exist:

In [55]:
all_files = [str(x) for x in pathlib.Path(data_root).rglob("*.nii*")]

for k, v in mapping.items():
    
    if not v:
        
        print("{}: This fieldmap is not intended for any files!".format(k))
        
        continue
        
    for fi in v:
        
        if any([fi in x for x in all_files]):
            
            print("{}: This fieldmap has a file".format(k))
            
        else:
            
            print("{}: The file this fieldmap is intended for doesn't exist".format(k))

/Users/ttapera/BBL/Projects/CuBIDS/notebooks/testdata/complete/sub-01/ses-phdiff/fmap/sub-01_ses-phdiff_acq-v4_phasediff.json: This fieldmap has a file
/Users/ttapera/BBL/Projects/CuBIDS/notebooks/testdata/complete/sub-01/ses-phdiff/fmap/sub-01_ses-phdiff_acq-v4_magnitude2.json: This fieldmap is not intended for any files!
/Users/ttapera/BBL/Projects/CuBIDS/notebooks/testdata/complete/sub-01/ses-phdiff/fmap/sub-01_ses-phdiff_dir-PA_epi.json: This fieldmap has a file
/Users/ttapera/BBL/Projects/CuBIDS/notebooks/testdata/complete/sub-01/ses-phdiff/fmap/sub-01_ses-phdiff_acq-v4_magnitude1.json: This fieldmap is not intended for any files!
/Users/ttapera/BBL/Projects/CuBIDS/notebooks/testdata/complete/sub-02/ses-phdiff/fmap/sub-02_ses-phdiff_acq-v4_phasediff.json: This fieldmap has a file
/Users/ttapera/BBL/Projects/CuBIDS/notebooks/testdata/complete/sub-02/ses-phdiff/fmap/sub-02_ses-phdiff_acq-v4_magnitude2.json: This fieldmap is not intended for any files!
/Users/ttapera/BBL/Projects/CuB

In the second check, we check that all files in `func`, `dwi` and `perf` have at least one fieldmap

TODO: `assign_fieldmaps()` function